# Skelettschätzer

In [ ]:
!pip install ipywidgets
!pip install tomli
!jupyter labextension install @jupyter-widgets/jupyterlab-manager

import os
import ipywidgets as widgets
from pathlib import Path
from IPython.display import display, clear_output, HTML

## 1. Auswählen von Videos
Aktuell sind die folgenden Videos als Import ausgewählt:

In [ ]:
# Data-Ordner finden
with open("../.env", "r") as f:
    env = [line.replace("\n", "").split("=") for line in f.readlines()]
env = {e[0]: e[1] for e in env}
data_folder = env["INPUT_FOLDER"]
input_folder = os.path.join(data_folder, "input")
input_folder_parsed = Path(input_folder)
# mp4-Videos suchen
files = os.listdir(input_folder)
mp4_files = list(filter(lambda f: f.endswith(".mp4"), files))
# YouTube-Videos suchen
if "input.txt" in files:
    with open(os.path.join(input_folder, "input.txt"), "r") as f:
        yt_links = [l.replace("\n", "") for l in f.readlines()]
else:
    yt_links = []

In [ ]:
# ---------- Widgets ----------

# Datei-Upload (nur mp4 zulassen)
upload_widget = widgets.FileUpload(
  accept='.mp4',
  multiple=True,
  description='MP4 hochladen'
)

# Mehrzeiliges Eingabefeld für URLs
url_textarea = widgets.Textarea(
  placeholder='Gib hier URLs ein (eine pro Zeile)',
  description='URLs:',
  layout=widgets.Layout(width='100%', height='100px')
)

# Button zum Löschen aller MP4-Dateien
delete_button = widgets.Button(
  description='Alle MP4-Dateien löschen',
  button_style='danger'
)

# Bestätigungsbutton
confirm_button = widgets.Button(
  description='Auswahl bestätigen',
  button_style='success'
)

# Ausgabe für die Dateiliste
file_list_output = widgets.Output()

# ---------- Funktionen ----------

def save_uploaded_files(change):
  for filename, fileinfo in upload_widget.value.items():
    file_path = input_folder_parsed / filename
    with open(file_path, 'wb') as f:
      f.write(fileinfo['content'])
  upload_widget.value.clear()
  update_file_list()

def update_file_list():
  with file_list_output:
    clear_output()
    mp4_files = list(input_folder_parsed.glob('*.mp4'))
    if mp4_files:
      display(HTML('<b>Vorhandene MP4-Dateien:</b>'))
      for file in mp4_files:
        display(HTML(f'📁 {file.name}'))
    else:
      display(HTML('<i>Keine MP4-Dateien im Ordner.</i>'))

def delete_all_mp4_files(b):
  for file in input_folder_parsed.glob('*.mp4'):
    file.unlink()
  update_file_list()

def confirm_selection(b):
  urls = url_textarea.value.strip().splitlines()
  print('✅ Bestätigt:')
  print(f'- {len(list(input_folder_parsed.glob("*.mp4")))} MP4-Dateien im Ordner')
  print(f'- {len(urls)} URLs eingegeben')
  for url in urls:
    print(f'  • {url.strip()}')

# ---------- Event-Handler verbinden ----------

upload_widget.observe(save_uploaded_files, names='value')
delete_button.on_click(delete_all_mp4_files)
confirm_button.on_click(confirm_selection)

# ---------- UI anzeigen ----------

display(widgets.VBox([
  upload_widget,
  url_textarea,
  delete_button,
  confirm_button,
  file_list_output
]))

# Initiale Anzeige
update_file_list()